# Enron project data exploration

## Explore dataset

In [2]:
import pickle

enron_data = pickle.load(open(r"C:\Users\dehaeth\Documents\Tools\Python\Oefeningen\Udacity\Machine learning intor\ud120-projects-master\final_project\final_project_dataset.pkl", "r"))

In [5]:
print enron_data["SKILLING JEFFREY K"]

{'salary': 1111258, 'to_messages': 3627, 'deferral_payments': 'NaN', 'total_payments': 8682716, 'exercised_stock_options': 19250000, 'bonus': 5600000, 'restricted_stock': 6843672, 'shared_receipt_with_poi': 2042, 'restricted_stock_deferred': 'NaN', 'total_stock_value': 26093672, 'expenses': 29336, 'loan_advances': 'NaN', 'from_messages': 108, 'other': 22122, 'from_this_person_to_poi': 30, 'poi': True, 'director_fees': 'NaN', 'deferred_income': 'NaN', 'long_term_incentive': 1920000, 'email_address': 'jeff.skilling@enron.com', 'from_poi_to_this_person': 88}


## Number of people, features and poi's

In [16]:
print "number of people: " + str(len(enron_data))
print "number of features: " + str(len(enron_data.values()[0]))
poi = 0
for person in enron_data.keys():
    if enron_data[person]['poi'] == True:
        poi +=1
        
print "number of poi's: " + str(poi)

number of people: 146
number of features: 21
number of poi's: 18


## Number of real poi's

In [27]:
poi_file = 0
poi_file_emails = 0

with open(r'C:\Users\dehaeth\Documents\Tools\Python\Oefeningen\Udacity\Machine learning intor\ud120-projects-master\final_project\poi_names.txt','r') as enronnamefile:
    for line in enronnamefile.readlines():
        if '(y)' in line:
            poi_file_emails += 1
            poi_file += 1
        elif '(n)' in line:
            poi_file += 1
              
            
print 'real pois: ' + str(poi_file)

real pois: 35


## Stock value of an individual

In [34]:
print enron_data['PRENTICE JAMES']['total_stock_value']
print enron_data['COLWELL WESLEY']['from_this_person_to_poi']

1095040
11


## Who took the largest cake

In [44]:
print "jeffrey skilling: " + str(enron_data['SKILLING JEFFREY K']['total_payments'])
print "ken lay: " + str(enron_data['LAY KENNETH L']['total_payments'])
print "andrew fastow: " + str(enron_data['FASTOW ANDREW S']['total_payments'])

jeffrey skilling: 8682716
ken lay: 103559793
andrew fastow: 2424083


## Checking salaries and email addresses

In [48]:
quantified_email = 0
quantified_salary = 0

for person in enron_data.keys():
    if enron_data[person]['salary'] != 'NaN':
        quantified_salary+=1
    if enron_data[person]['email_address'] != 'NaN':
        quantified_email+=1    
        
print quantified_email
print quantified_salary     

111
95


## Missing financial info

In [56]:
missing_payment = 0

for person in enron_data.keys():
    if enron_data[person]['total_payments'] == 'NaN':
        missing_payment+=1   
        
print missing_payment
print float(missing_payment) / float(len(enron_data.keys())) * 100
print missing_payment + 10
print float(missing_payment + 10) / float(len(enron_data.keys()) + 10) * 100

21
14.3835616438
31
19.8717948718


## Missing financial info POI

In [59]:
missing_payment = 0
poi = 0

for person in enron_data.keys():
    if enron_data[person]['poi'] == True:
        poi += 1
        if enron_data[person]['total_payments'] == 'NaN':
            missing_payment+=1   
        
        
print missing_payment
print float(missing_payment) / float(poi) * 100
print missing_payment + 10
print (float(missing_payment + 10) / float(poi + 10)) * 100

0
0.0
10
35.7142857143


# Basic machine learning

## Splitting tools

In [47]:

import numpy as np

def featureFormat( dictionary, features, remove_NaN=True, remove_all_zeroes=True, remove_any_zeroes=False, sort_keys = False):

    return_list = []

    # Key order - first branch is for Python 3 compatibility on mini-projects,
    # second branch is for compatibility on final project.
    if isinstance(sort_keys, str):
        import pickle
        keys = pickle.load(open(sort_keys, "rb"))
    elif sort_keys:
        keys = sorted(dictionary.keys())
    else:
        keys = dictionary.keys()

    for key in keys:
        tmp_list = []
        for feature in features:
            try:
                dictionary[key][feature]
            except KeyError:
                print "error: key ", feature, " not present"
                return
            value = dictionary[key][feature]
            if value=="NaN" and remove_NaN:
                value = 0
            tmp_list.append( float(value) )

        # Logic for deciding whether or not to add the data point.
        append = True
        # exclude 'poi' class as criteria.
        if features[0] == 'poi':
            test_list = tmp_list[1:]
        else:
            test_list = tmp_list
        ### if all features are zero and you want to remove
        ### data points that are all zero, do that here
        if remove_all_zeroes:
            append = False
            for item in test_list:
                if item != 0 and item != "NaN":
                    append = True
                    break
        ### if any features for a given data point are zero
        ### and you want to remove data points with any zeroes,
        ### handle that here
        if remove_any_zeroes:
            if 0 in test_list or "NaN" in test_list:
                append = False
        ### Append the data point if flagged for addition.
        if append:
            return_list.append( np.array(tmp_list) )

    return np.array(return_list)


def targetFeatureSplit( data ):


    target = []
    features = []
    for item in data:
        target.append( item[0] )
        features.append( item[1:] )

    return target, features
